In [87]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [88]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-04-03 08:02:49--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M   108MB/s    in 0.2s    

2025-04-03 08:02:49 (108 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [89]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [90]:
# add your code here - consider creating a new cell for each section of code
print(df_books.head())



         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
4       E. J. W. Barber  


In [49]:
print(df_ratings.head())

     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0


In [91]:
print(f"Unique users: {df_ratings['user'].nunique()}")
print(f"Unique books: {df_ratings['isbn'].nunique()}")

Unique users: 105283
Unique books: 340556


# Data Cleaning

In [92]:
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Remove users with fewer than 200 ratings
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Remove books with fewer than 100 ratings
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]



#Book_user Matrix pivot table

In [93]:
df_merged = pd.merge(df_ratings, df_books, on='isbn')

book_user_matrix = df_merged.pivot_table(
    index='title',    # use book title instead of ISBN for easier interpretation
    columns='user',
    values='rating'
).fillna(0)


In [94]:
book_user_matrix.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
book_user_matrix.shape

(673, 888)

#Knn model

In [96]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_matrix.values)

book_titles = list(book_user_matrix.index)


In [97]:
book_titles[:5]

['1984',
 '1st to Die: A Novel',
 '2nd Chance',
 '4 Blondes',
 'A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash']

#Test

In [98]:
def get_recommends(book_title):
    if book_title not in book_titles:
        return [book_title, []]

    # Get the index of the book
    book_idx = book_titles.index(book_title)

    # Find the 6 nearest neighbors (including the book itself)
    distances, indices = model.kneighbors([book_user_matrix.iloc[book_idx]], n_neighbors=6)
    print(distances, indices)
    # Build the recommendations list (skip index 0 because it's the book itself)
    recs = []
    for i in range(1, len(indices[0])):
        similar_title = book_titles[indices[0][i]]
        distance = float(distances[0][i])
        recs.append([similar_title, distance])

    # reverse to show them in ascending order
    recs.reverse()
    return [book_title, recs]


In [99]:
# Test the function
print(get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))"))


[[0.         0.7234864  0.7677075  0.7699411  0.77085835 0.8016211 ]] [[654 539 240 597 614 243]]
["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]


In [100]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False

  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]

  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

[[0.         0.7234864  0.7677075  0.7699411  0.77085835 0.8016211 ]] [[654 539 240 597 614 243]]
["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
[[0.         0.7234864  0.7677075  0.7699411  0.77085835 0.8016211 ]] [[654 539 240 597 614 243]]
You passed the challenge! 🎉🎉🎉🎉🎉
